In [1]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd


In [2]:

n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A.csv")
pi = pd.read_csv("pi.csv")
p = pd.read_csv("p.csv")

A = np.array(A)
pi = np.array(pi)
pi = pi.T
b = np.array([1000 for _ in range(m)])  


OffLine

In [3]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)

prob.solve()
x.value


print(x.value)
print(prob.value)
opv1 = prob.value

x = x.value
x = np.where(x >= 0.5, 1, 0)

[9.99999998e-01 3.20223562e-11 9.99999998e-01 ... 1.39041461e-09
 3.95575732e-10 3.98576014e-09]
282270.0744458285


K = 50

In [4]:
k = 50
steps = n/k
Anow = A[:, 0:k]
Anow = Anow.T
# print(Anow.shape)
piNow = (pi.T)[0:k]
piNow = np.reshape(piNow, (k,))
bNow = b*(float(k/n))
bNow = bNow.T

temp = m+k
y = cp.Variable(temp)
e = np.ones(k)
objective = cp.Minimize(bNow@(y[:m]) + e@(y[m:]))
constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
prob = cp.Problem(objective, constraints)

prob.solve()
P = y[:m].value

AT = A.T


xp50 = []
xk50 = []
for i in range(n):
    cri = (AT[i].dot(P))
    if(pi[0][i] > cri):
        xp50.append(1)
    else:
        xp50.append(0)  # xt(p) 10000
    if i < k:
        xk50.append(0)  #50


In [5]:
def MAE_Loss(act, pred):
    diff = act - pred
    diff_abs = np.absolute(diff)
    mae = diff_abs.mean()
    return mae

In [6]:
y[:m].value

array([44.08131118, 34.73590639, 14.2834981 , 49.16637156, 25.32845307,
       26.23525563,  9.9796159 , 25.22667587, 13.64073604, 39.06147426])

In [7]:
a = y[:m].value.reshape(-1,1)
MAE_Loss(a, p)

0    0.202678
dtype: float64

In [8]:
for t in range(k,n):
    test = 1
    for j in range(m):
        temp1 = A[j,t]*xp50[t]
        temp2 = b[j] - (A[j, :t]).dot(np.array(xk50))
        if temp1 > temp2:
            test = 0
            break
    if test == 1:
        xk50.append(xp50[t])
    else:
        xk50.append(0)

In [9]:
xk50 = np.array(xk50)
opv2 = (pi.dot(xk50))[0]
rateK50 = opv2/opv1

rateK50

0.7187189925298438

k100

In [10]:
k = 100
steps = n/k
Anow = A[:, 0:k]
Anow = Anow.T
# print(Anow.shape)
piNow = (pi.T)[0:k]
piNow = np.reshape(piNow, (k,))
bNow = b*(float(k/n))
bNow = bNow.T


y = cp.Variable((m+k))
e = np.ones(k)
objective = cp.Minimize(bNow@(y[:m]) + e@(y[m:]))
constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
prob = cp.Problem(objective, constraints)

prob.solve()
P = y[:m].value

AT = A.T


xp100 = []
xk100 = []

for i in range(n):
    cri = (AT[i].dot(P))
    if(pi[0][i] > cri):
        xp100.append(1)
    else:
        xp100.append(0)  # xt(p) 10000
    if i < k:
        xk100.append(0)  #50

for t in range(k,n):
    test = 1
    for j in range(m):
        temp1 = A[j,t]*xp100[t]
        temp2 = b[j] - (A[j:j+1, :t]).dot(np.array(xk100))
        if temp1 > temp2:
            test = 0
            break
    if test == 1:
        xk100.append(xp100[t])
    else:
        xk100.append(0)
# for i in range(k,n):
#     cri = (AT[i].dot(P))
#     if(pi[0][i] > cri):
#         xp100.append(1)
#     else:
#         xp100.append(0)

#     if i < k:
#         xk100.append(0)
#         continue
#     test = 1
#     for j in range(m):
#         temp1 = A[j,i]*xp100[i]
#         temp2 = b[j] - (A[j:j+1, :i]).dot(np.array(xk100))
#         if temp1 > temp2:
#             test = 0
#             break
#     if test == 1:
#         xk100.append(xp100[i])
#     else:
#         xk100.append(0)


In [11]:
a = y[:m].value.reshape(-1,1)
MAE_Loss(a, p)

0    0.086771
dtype: float64

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [12]:
xk100 = np.array(xk100)
opv3 = (pi.dot(xk100))[0]
rateK100 = opv3/opv1

rateK100


0.8890096880281994

k200

In [13]:
k = 200
steps = n/k
Anow = A[:, 0:k]
Anow = Anow.T
# print(Anow.shape)
piNow = (pi.T)[0:k]
piNow = np.reshape(piNow, (k,))
bNow = b*(float(k/n))
bNow = bNow.T


y = cp.Variable((m+k))
e = np.ones(k)
objective = cp.Minimize(bNow@(y[:m]) + e@(y[m:]))
constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
prob = cp.Problem(objective, constraints)

prob.solve()
P = y[:m].value

AT = A.T
# print("----------")
# print(AT[0])

xp200 = []
xk200 = []

for i in range(n):
    cri = (AT[i].dot(P))
    if(pi[0][i] > cri):
        xp200.append(1)
    else:
        xp200.append(0)  # xt(p) 10000
    if i < k:
        xk200.append(0)  #50

for t in range(k,n):
    test = 1
    for j in range(m):
        temp1 = A[j,t]*xp200[t]
        temp2 = b[j] - (A[j:j+1, :t]).dot(np.array(xk200))
        if temp1 > temp2:
            test = 0
            break
    if test == 1:
        xk200.append(xp200[t])
    else:
        xk200.append(0)
        
# for i in range(n):
#     cri = (AT[i].dot(P))
#     if(pi[0][i] > cri):
#         xp200.append(1)
#     else:
#         xp200.append(0)

#     if i < k:
#         xk200.append(0)
#         continue
#     test = 1
#     for j in range(m):
#         temp1 = A[j,i]*xp200[i]
#         temp2 = b[j] - (A[j:j+1, :i]).dot(np.array(xk200))
#         if temp1 > temp2:
#             test = 0
#             break
#     if test == 1:
#         xk200.append(xp200[i])
#     else:
#         xk200.append(0)


In [14]:
a = y[:m].value.reshape(-1,1)
MAE_Loss(a, p)

0    0.067249
dtype: float64

In [15]:
xk200 = np.array(xk200)
opv4 = (pi.dot(xk200))[0]
# print(opv1)
# print(type(opv1))
rateK200 = opv4/opv1
print(xk200)
rateK200

[0 0 0 ... 0 0 0]


0.9303039098217225